## Create Mike's dictionary of Morton codes with start and end points for the 512^3

In [ ]:
desired_cube_side = 512
chunk_size = 64
array_cube_side = 2048

raw_ncar_folder_path = '/home/idies/workspace/turb/data02_02/ncar-high-rate-fixed-dt'

dest_folder_name = "sabl2048b" # B is the high-rate data
write_type = "prod" # or "back"
timestep_nr = 1
# timestep_range = range(1) # Ned's new High-rate fixed-dt has only 5 timesteps

In [ ]:
%cd /home/idies/workspace/Storage/ariel4/persistent/ncar-zarr-code

In [ ]:
from utils import write_utils

import xarray as xr
# from utils import write_utils
import os

### Get target Folder list

<font color="orange">Don't delete the CD cell!</font>

In [ ]:
%cd /home/idies/workspace/turb/data02_02/ncar-high-rate-fixed-dt

In [ ]:
merged_velocity = xr.open_dataset(raw_ncar_folder_path + "/jhd." + str(timestep_nr).zfill(3) + ".nc")

In [ ]:
# Split 2048^3 into smaller 512^3 arrays
dims = [dim for dim in merged_velocity.dims]
dims.reverse() # use (nnz, nny, nnx) instead of (nnx, nny, nnz)

smaller_groups, range_list = write_utils.split_zarr_group(merged_velocity, desired_cube_side, dims)

In [ ]:
# Given up in favor of Ryan's node coloring technique
#     z_order = write_utils.morton_order_cube(cube_side=4)

node_assignments = write_utils.node_assignment(cube_side=4)

In [ ]:
range_list[:15]

In [ ]:
folders=write_utils.list_fileDB_folders()

# Avoiding 7-2 and 9-2 - they're too full as of May 2023
folders.remove("/home/idies/workspace/turb/data09_02/zarr/")
folders.remove("/home/idies/workspace/turb/data07_02/zarr/")

for i in range(len(folders)):
    folders[i] += dest_folder_name + "_" + str(i + 1).zfill(2) + "_" + write_type + "/"

# folders[:5]

# Create top-level dirs

# for folder_path in folders:
#     os.makedirs(folder_path, exist_ok=False)

In [ ]:
sorted_morton_list = [] # Sorting by Morton code to be consistent with Isotropic8192
for i in range(len(range_list)):            
    min_coord = [a[0] for a in range_list[i]]
    max_coord = [a[1] - 1 for a in range_list[i]]
            
    sorted_morton_list.append((write_utils.morton_pack(array_cube_side, min_coord[0], min_coord[1], min_coord[2]), write_utils.morton_pack(array_cube_side, max_coord[0], max_coord[1], max_coord[2])))
        # (write_utils.morton_pack(array_cube_side, min_coord[2], min_coord[1], min_coord[0]), write_utils.morton_pack(array_cube_side, max_coord[2], max_coord[1], max_coord[0]))

        
sorted_morton_list = sorted(sorted_morton_list)
sorted_morton_list[:5]

In [ ]:
mike_dict = {}
for i in range(len(range_list)):            
    min_coord = [a[0] for a in range_list[i]]
    max_coord = [a[1] - 1 for a in range_list[i]]
            
    mike_dict[dest_folder_name + str(i + 1).zfill(2)] = sorted_morton_list[i]
#         (morton_pack(array_cube_side, min_coord[0], min_coord[1], min_coord[2]), morton_pack(array_cube_side, max_coord[0], max_coord[1], max_coord[2]))
        # (write_utils.morton_pack(array_cube_side, min_coord[2], min_coord[1], min_coord[0]), write_utils.morton_pack(array_cube_side, max_coord[2], max_coord[1], max_coord[0]))
        
    # Ordered 2-1-0 bcs. dims are nnz-nny-nnx
    
#     print(mike_dict[dest_folder_name + str(i + 1).zfill(2)])

In [ ]:
mike_dict

In [ ]:
def flatten_3d_list(lst_3d):
    return [element for sublist_2d in lst_3d for sublist_1d in sublist_2d for element in sublist_1d]

def search_dict_by_value(dictionary, value):
    for key, val in dictionary.items():
        if val == value:
            return key
    return None  # Value not found in the dictionary

In [ ]:
flattened_node_assgn = flatten_3d_list(node_assignments)
print(flattened_node_assgn)

In [ ]:
len(range_list)

In [ ]:
cubes = range_list

In [ ]:
for i in range(len(range_list)):
    min_coord = [a[0] for a in range_list[i]]
    max_coord = [a[1] - 1 for a in range_list[i]]
    
    # nnz - nny - nnx
    morton = (write_utils.morton_pack(array_cube_side, min_coord[2], min_coord[1], min_coord[0]), write_utils.morton_pack(array_cube_side, max_coord[2], max_coord[1], max_coord[0]))
    
    chunk_name = search_dict_by_value(mike_dict, morton)
    
    idx = int(chunk_name[-2:].lstrip('0'))
    
    filedb_index = flattened_node_assgn[idx - 1] - 1
    
    destination = os.path.join(folders[filedb_index], dest_folder_name + str(idx).zfill(2) + "_" + str(timestep_nr).zfill(3) + ".zarr")
    print(destination)
    
    current_array = cubes[i]
    print(current_array, '|' , morton)

In [ ]:
%cd /home/idies/workspace/Storage/ariel4/persistent/ncar-zarr-code/zarr_writing

In [ ]:
import dill

pickle_file = './filename_morton_ranges.pickle'

# the dictionary that will be written to the pickle file. empty dictionary in this example, but you would fill it with the data.
filename_morton_map = mike_dict

with open(pickle_file, 'wb') as pickled_filepath:
    dill.dump(filename_morton_map, pickled_filepath)